<a href="https://colab.research.google.com/github/samyzaf/notebooks/blob/main/heat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2D Heat Equation Simulator using FDM  
<small>version: 20.12.2024a</small>

* **FDM** stands for **Finite Difference Method**.
* This is a numerical method for solving
  differential equations, which is explained in details at the course and in the lecture notes.
* Additional details are provided in the PDF
  [**Project Booklet**](https://samyzaf.com/fdm/fdm.pdf).
* Heat equation simulator Python code was inspired by
  a blog of **G. Nervadof**:  
  https://levelup.gitconnected.com/solving-2d-heat-equation-numerically-using-python-3334004aa01a

* You need to install our
  [**fdmtools Python Package**](https://samyzaf.com/fdmtools.zip).
* Please run the next code cell to install
  and export these tools to this notebook.

In [ ]:
%pip install -q https://samyzaf.com/fdmtools.zip
from fdmtools.heat2 import *

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
fdmtools version 5


* We show how to solve an example of the heat equation over a
  rectangular plate (in fact a **square**) which has
  the following general form

* Additional similar problems can be solved in the same manner.
  
<IMG src="https://samyzaf.com/fdm/heat2b.jpg" width=800 align="center"/>

## **Finite Difference Method (FDM)**

* The finite difference method is based on the idea
  that 2D plane or 3D space can be represented as
  a collection of discrete pieces of small elements
  of area (2D) or volume (3D).  

  <IMG src="https://samyzaf.com/fdm/cover4.jpg" width=800 align="center"/>

* In our case, we need the 3D modelling, with the
  following parameters.

## **Basic parameters**
As mentioned, we will use a square plate (**L=a=b**)
with the following FDM parameters
* **L** = Plate Length = Width = Height
* **N** = Grid division Number (dx = dy = L/N)
* **alpha** = Heat physical constant
* **dt** = Time grid unit
* **fps** = Frames Per Second (for video)
* **seconds** = Video Simulation time
  (also video duration)
* **num_frames** = Total number of **time frames** in our simulation
* **framestep** = 10 - for our video,
  we take 1 frame from each 10 frames.
  You may change this number in your tests.
* **dx = dy =** Plate Length grid units (L/N)
* **U** = 3D **[Numpy](https://numpy.org)** Array

### Notes
* Note the difference between **num_frames** and **frames**.
* **num_frames** is the total number of frames we plan to make.
* **frames** is a range object which by default consists
  of all frames from 1 to **num_frames**, but in other
  situations we may want to skip some frames and take only
  a subset. We will do this in the wave equation case.

## **Example 1: simple square plate `LxL`, `L=4`**
* To make it as simple as possible we will work on
  a square domain with length **`L=a=b=4`**.

* Exact coordinates: **`D = [0,4]x[0,4]`**

* We will use the **Finite Difference Method (FDM)**
  to solve the following problem on a 4x4 plate  
    
$\qquad
\left\{
\begin{array}{ll}
u_t = 0.1(u_{xx} + u_{yy}), &\quad 0<x<4,\quad 0<y<4,\quad 0\leq t<\infty
\\
u(x,y,0) = 0,  &\quad 0<x<4,\quad 0<y<4,\quad t=0
\\
u(x,0,t) = 100\,\sqrt[3]{x/4},  &\quad 0\leq x\leq 4,\quad 0\leq t<\infty
\\
u(x,4,t) = 100(0.7 + 0.3\sin\frac{5\pi x}{4}),  &\quad 0\leq x\leq 4,\quad 0\leq t<\infty
\\
u(0,y,t) = 100\sqrt{y/4},  &\quad 0\leq y\leq 4,\quad 0\leq t<\infty
\\
u(4,y,t) = 0,  &\quad 0\leq y\leq 4,\quad 0\leq t<\infty
\end{array}
\right.
$

* Here is our specific **FDM Parameter Plan** for this problem:

In [ ]:
L = 4.0                   # Length/width of plate
N = 50                    # Grid division size
alpha = 0.1               # Physical heat constant
seconds = 20              # Simulation time (also video duration)
fps = 30                  # Frames per second (for video simulation)
framestep = 10            # pick one video frame from every 10 time frames
num_frames = int(seconds*framestep*fps)  # Total number of time frames
dx = dy = L/N             # Step sizes = x,y grid units
dt = dx**2 / (4 * alpha)  # An automatic choice for dt! (see CFL condition in project booklet)
T = seconds * fps * dt    # Physical time (the real time)
U = np.zeros((N+1, N+1, num_frames))   # 3D Numpy Array - this holds our FDM grid!

print(f"dx = {dx}")
print(f"dt = {dt} (auto choice)")

dx = 0.08
dt = 0.016 (auto choice)


## **Boundary Conditions**

* Our [**fdmtools package**](https://samyzaf.com/fdmtools.zip)
  allows us to specify boundary conditions easily!

In [ ]:
f  = lambda x,y: 0                               # Initial temperature in plate internal points (t=0)
f1 = lambda x: 100 * (x/L) ** (1/3)              # Constant temperature on lower side
f2 = lambda y: 100 * (0.7 + 0.3*sin(5*pi*y/L))   # Constant temperature on upper side
g1 = lambda y: 100 * (y/L)**(1/2)                # Constant temperature on left side
g2 = lambda y: 0                                 # Constant temperature on right side

* We start with the initial
  temperature `f(x,y)` on the whole plate at
  time frame t=0.

* Then we set the boundary conditions on the plate sides
  using 4 functions `f1, f2, g1, g2`.
  These conditions are constant for all time $t$.

* See details in the following figure.  

  <IMG src="https://samyzaf.com/fdm/heat5e.jpg" width=540 align="center"/>

* The Python **lambda** command is a simple mechanism
  for creating one line functions.

* In most simple cases it is enough, but in more complex
  cases, you may need to use the standard **def** mechanism
  for defining a Python function.

## **Heat Equation Solver**

* Our **FDM grid** is modelled by the
  3D [Numpy](https://numpy.org) Array **U**
  which we defined above.

* To get **N** sub intervals division, we need
  **N+1** nodes. Therefore, the shape of
  **U** must be:
  **(N+1) x (N+1) x num_frames**.

* Mathematically, **U** is a 3D matrix.
  However, note that software arrays behave a little
  differently than in Math.

* If you need to, use **U[i,j,k]** to access pixel **(i,j)**
  on the plate at time frame **k**.

* The value **U[i,j,k]** is destined to be
  **FDM** approximation to the solution $u(x_i,y_j,t_k)$
  on the grid nodes.

* It takes 5 lines of Python code to implement the recursive
  formula for solving the discrete heat equation.

* The following **Solve** method is part of our
  [**fdmtools Python package**](https://samyzaf.com/fdmtools.zip),
  which includes the Python code needed to run this
  Colab notebook.

  <IMG src="https://samyzaf.com/fdm/code_heat2d1.jpg" width=700 align="center"/>
* This method simulates the temperature frames on
  **all plate points** for the **full time duration**,
  and saves them in the array **U**.
* For every pixel **`(i,j)`**, and every time frame **`k`**,
  **`U[i,j,k]`** is the temperature of the plate at pixel
  **`(i,j)`**, at time frame **`k`**.

<!--
* We use the **[Numba package](https://numba.pydata.org)
  just in time compiler** for speed enhacement
  (thanks to Intel/Nvidia support).
* It translates the Python code to C and
  gets about 1000x speed enhancement.
-->

* The validity of this **numerical** method will be presented
  and explained in the main project booklet.
  Under certain conditions it yields very good
  approximation to the real solution.

* The 3D array **U** is obtained by selecting a finite
  number of discrete points $(x_i,y_j,t_k)$ from
  the domain **D** of the infinite continuous
  function $u(x,y,t)$.
  * **$\mathbf{x_i}$ = x0 + i*dx/N**
  * **$\mathbf{y_j}$ = y0 + j*dy/N**
  * **$\mathbf{t_k}$ = t0 + k*dt**
  * **U[i,j,k] $\;\mathbf{\leftrightarrow}\;$ $\mathbf{u(x_i,y_j,t_k)}$**
  
* Each **node** is encoded by the array key **(i,j,k)**
  of 3 integers, which is easier to manipulate by computer algorithms.

* We thank again
  [**G. Nervadof**](https://levelup.gitconnected.com/solving-2d-heat-equation-numerically-using-python-3334004aa01a),
  for his excellent blog from which we borrowed most of
  the code for our
  [**package**](https://samyzaf.com/fdmtools.zip).

* We use the [matplotlib animation package](https://matplotlib.org/stable/users/explain/animations/animations.html)
  for generating an **mp4** video from the sequence of
  temperature frames.

<!--
* Note: in software, array **U[i,j,k]** indices
  are somewhat reversed.
  * The index **i** stands for **row number** and
     therefore corresponds to **y**!
  * The index **j** stands for columns number and
     therefore corresponds to **x**!
  * Therefore directly manipulating Numpy arrays is
     suited for experienced programmers, so we have created easier interface for specifying our initial
     and boundary conditions as above.
-->

* Now we are ready to run the simulation.

In [ ]:
video_file = "heat_ex1.mp4"   # Video file name
video_width = 540             # Video display width (pixels)
figure_width = 5              # Figure real width (inches)
dpi = 150                     # Video resolution (dots per inch)
fontsize = 9                  # Text font size

RunSimulation()

Solve time: 1.726
Umin = 0.0
Umax = 100.0
Animating: 100%   
Time: 194.24 seconds
Done!
Simulation run time = 20 seconds
Physical run time = 9.6 seconds
Number of frames = 6000
Frames per second = 30
dx = dy = 0.08
dt = 0.016
Playing video file


## Comments
* **frames** = Video frames range (integer list from 0 to num_frames-1)
* **pbar** = Progress Bar object for monitoring the progress of creating our video simulation.
* The **boundary_conditions** method takes care of
  initializing the Array **U** with the correct boundary
  values.
  Converting x/y/t values to i/j/k Numpy array indices
  is not intuitive and at first stage can be skipped.
  Students interested in how this is done can
  download the
  **[fdmtools package](https://samyzaf.com/fdmtools-3.zip)**
  and browse the Python code for all the details.
* Note that *video run time* is
  not identical to real *physical run time*!
* The physical heat phenomenon may take a very short time,
  but the video must slow it down so we can observe it slowly.
  In other cases it might be too slow, and the simulation will speed it up.
* Simulation (or video) run time is defined by
  the **seconds** parameter.
* Physical run time can be calculated by multiplying
  the number of video frames by **dt**:

In [ ]:
print(f"Simulation run time = {seconds} seconds")
print(f"Physical run time = {num_frames/framestep * dt} seconds")

Video file = heat_ex1.mp4
You may download it to your local device by: file_download(video_file)
Simulation run time = 20 seconds
Physical run time = 9.6 seconds


* Note that the number of video frames is smaller than
  the total number of frames! Since we sample
  only one video frame per 10 time frames.
* To generate higher resolution videos, you may need
  to increase the size of the parameters
  **`N`**, and **seconds**.
* You may try **`N=100`** or **`N=200`**, but it will
  take much more time to generate the video!

* You may also want to increase the video time.
  You may try **`seconds=30`** or **`seconds=60`**.

# CONCLUSION AND CODE SUMMARY
* For convenience, we have gathered all the code
  inside **one code cell**, which you can run
  **repeatedly** without restarting the notebook.

* You can copy and paste this cell for
  experimenting with more cases.

In [ ]:
L = 4.0                   # Length/width of plate
N = 100                   # Grid division size
alpha = 0.1               # Physical heat constant
seconds = 2              # Simulation time (also video duration)
fps = 30                  # Frames per second (for video simulation)
framestep = 10            # pick one video frame from every 10 time frames
num_frames = int(seconds*framestep*fps)  # Total number of time frames
dx = dy = L/N             # Step sizes = x/y grid units
dt = dx**2 / (4 * alpha)  # A better automatic choice for dt!
print(f"dt = {dt} (auto choice)")
T = seconds * fps * dt    # Physical time (the real time)

# 3D Numpy array
U = np.empty((N+1, N+1, num_frames))

# Boundary conditions
f  = lambda x,y: 0
f1 = lambda x: 100 * (x/L) ** (1/3)
f2 = lambda x: 100 * (0.7 + 0.3*sin(5*pi*x/L))
g1 = lambda y: 100 * (y/L)**(1/2)
g2 = lambda y: 0

# Graphic parameters
video_file = "heat_ex1.mp4"    # Video file name
video_width = 540              # Video display width (pixels)
figure_width = 5               # Figure width size (inches)
dpi = 150                      # Video resolution (dots per inch)
fontsize = 8                   # Text font size

math_text = [                  # LaTeX Math formula text for video
    r"$u_{t} = \alpha (u_{xx} + u_{yy})$",
    r"$u(x,0,t) = 100(0.7 + 0.3\sin\frac{5\pi x}{4})$",
    r"$u(x,4,t) = 0$",
    r"$u(0,y,t) = 100\sqrt{y/4}$",
    r"$u(4,y,t) = 0$"
]

RunSimulation()

dt = 0.004 (auto choice)
Solve time: 0.244
Umin = 0.0
Umax = 100.0
Animating: 100%   
Time: 23.62 seconds
Done!
Simulation run time = 2 seconds
Physical run time = 0.24 seconds
Number of frames = 600
Frames per second = 30
dx = dy = 0.04
dt = 0.004
Playing video file


Video file = heat_ex1.mp4
You may download it to your local device by: file_download(video_file)


* You can download your video file to your local
  computer by running the following command.

In [ ]:
#file_download("heat_ex2.mp4")

* You can also plot particular heat maps at a particular
  time frame using the following code.

In [ ]:
#plot_heatmap(U[:,:,18], 18)
#plt.show()

## **Example 2**
* We can copy the previous example code and easily make
  a new example by modifying a few parameters.

* This time we start with a non-trivial $u(x,y,0) = f(x,y)$
  condition.

* The other boundary conditions are much simpler.

In [ ]:
L = 3.0                   # Length of plate
N = 100                   # Grid division size
alpha = 0.02              # Physical heat constant
seconds = 30              # Simulation time (also video duration)
fps = 30                  # Frames per second (for video simulation)
framestep = 1             # pick one video frame from every 10 time frames
num_frames = int(seconds*framestep*fps)  # Total number of time frames
dx = dy = L/N             # Step sizes = x/y grid units
dt = dx**2 / (4 * alpha)  # A better automatic choice for dt!
print(f"dt = {dt} (auto choice)")
T = seconds * fps * dt    # Physical time (the real time)

# 3D Numpy array
U = np.empty((N+1, N+1, num_frames))

# Boundary conditions
f  = lambda x,y: 5*(x + y) / L
f1 = lambda x: x/L
f2 = lambda x: 0
g1 = lambda y: y/L
g2 = lambda y: 0

# Graphic parameters
text_x = 0.00                 # x coordinate for text
text_y = 1.08                 # y coordinate for text
math_x = 0.65                 # x coordinate for math text
video_file = "heat_ex2.mp4"   # Video file name
video_width = 540             # Video display width
figure_width = 5              # matplotlib figure width (inches)
dpi = 120                     # Video resolution (dots per inch)
fontsize = 8
linespacing = 1.5

math_text = [                 # LaTeX Math formula text for video
    r"$u_{t} = \alpha (u_{xx} + u_{yy})$",
    r"$u(x,0,t) = \frac{x}{3}$",
    r"$u(x,3,t) = 0$",
    r"$u(0,y,t) = \frac{y}{3}$",
    r"$u(3,y,t) = 0$",
]

RunSimulation()

dt = 0.01125 (auto choice)
Solve time: 0.407
Umin = 0.0
Umax = 9.899999999999999
Animating: 100%   
Time: 289.47 seconds
Done!
Simulation run time = 30 seconds
Physical run time = 10.125 seconds
Number of frames = 900
Frames per second = 30
dx = dy = 0.03
dt = 0.01125
Playing video file


Video file = heat_ex2.mp4
You may download it to your local device by: file_download(video_file)


## Final Notes for Python Coders

* This section is intended only for Python coders.

* If you are a Python coder, you may want to download the
  **[fdmtools package](https://samyzaf.com/fdmtools-3.zip)**
  and read the code and use it for other purposes.
  It is a small readable package that contains important
  details regarding the other Python that we use,
  which is very useful to know.

* For Python users, it is worth looking at the
  **boundary_conditions** method in order to understand
  how the pure math expressions that we use above for these
  conditions, are converted to our discrete Numpy array **U**
  <IMG src="https://samyzaf.com/fdm/code_heat_bc.jpg" width=700 align="center"/>

* The **":"** symbol in Python stands for the
  default full range **"0:N"**.

* So, for example, the expression **`U[:,:,0]`** is
  actually **`U[0:N, 0:N, 0]`**
  which is the matrix for the temperature
  at time frame **`k=0`**.
* The expression **`U[0,:,:]`** is actually
  **`U[0, 0:N, 0:num_frames]`** which represents the
  matrix for the bottom side of the plate over all time frames.

* Another issue is that Numpy arrays are implemented as
  nested lists, and that's why we see nested lists in the last
  5 lines of the code above.
  For example, the initial temperature frame (t=0) is
  obtained **f(x,y)** on each row, and then taking the
  list of these lists.

* The main reason that we decided to hide these details inside
  the package is because the reverse order of x/y and i/j in
  Python arrays, which is potentially confusing at first.
  One must map x to j (column index),
  and y to i (row index).
  Understanding the internal works of a Numpy array is not
  required for doing the project,
  and can be deferred to a later stage after gaining enough
  coding experience.